# Fusion

The following code is to be used on the numpy arrays derived from the 3D CNN and SGCNN models

In [47]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torchvision import transforms, utils
import numpy as np
import pandas as pd
from sklearn.metrics import *
from scipy.stats import *
import matplotlib.pyplot as plt
from collections import OrderedDict

In [48]:
def save_model(epochs, model, optimizer, criterion):
    """
    Function to save the trained model to disk.
    """
    print(f"Saving final model...")
    torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'models/fusion.pth')

In [49]:
class FusionDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, numpy_file,true_file):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # self.feat1 = np.load(numpy_file1)
        # self.feat2 = np.load(numpy_file2)
        # self.feat = np.concatenate([self.feat1,self.feat2],axis=1)
        self.feat = numpy_file
        self.y = true_file

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):

        return self.feat[idx],self.y[idx]

In [51]:
class ATOMFusionModel(nn.Module):
    # Based on the ATOM Fusion Model
    def __init__(self):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc11 = nn.Linear(10, 5) 
        self.fc12 = nn.Linear(6, 5) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()
        self.bn1 = nn.BatchNorm1d(5)
        self.bn2 = nn.BatchNorm1d(10)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.5)

        # Linear function (readout)
        self.fc2 = nn.Linear(26, 10) 
        
        self.fc3 = nn.Linear(10, 1) 

    def forward(self, x):
        feat = x
        feat1 = feat[:,:10]
        feat2 = feat[:,10:]
        # Linear function  # LINEAR
        hidden1 = self.fc11(feat1)
        hidden2 = self.fc12(feat2)
        hidden1 = self.lrelu(self.bn1(self.drop(hidden1)))
        hidden2 = self.lrelu(self.bn1(self.drop(hidden2)))
        

        # Non-linearity  # NON-LINEAR
        concat = torch.cat((feat1, hidden1, feat2, hidden2), 1)
        
        hidden3 = self.fc2(concat)
        hidden3 = self.bn2(self.relu(hidden3))
        
        hidden4 = self.fc3(hidden3)
        
        
        out = hidden4
        return out

Define the classical feedforward fusion network. Composed of a couple of feedforward layers followed by a ReLU activation.

In [52]:
od = OrderedDict()

num_layers = 4
for i in range(num_layers):
    od[str(i)] = nn.Linear(16, 16)

od[str(num_layers)] = nn.Linear(16, 1)
od[str(num_layers+1)] = nn.ReLU()

In [53]:
learning_rate = 0.0001
criterion = nn.MSELoss()

optim_param = [1, 0.002, 0.9, 0.999, 1e-08]
model = nn.Sequential(
    od
)

model = FeedforwardNeuralNetModel()
optimizer = torch.optim.Adam(model.parameters(), lr=optim_param[1],eps = optim_param[4])  
optimizer = torch.optim.Adam(model.parameters(), lr=optim_param[1], betas=(optim_param[2],optim_param[3]),eps = optim_param[4])

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.99)

num_epochs = 10000
batch_size=50


Defining the training and testing datasets

In [54]:
refined_val = np.concatenate([np.load('refined_val_fc10.npy'),np.load('refined_val_fc6_new.npy')],axis=1)
refined_train = np.concatenate([np.load('refined_train_fc10.npy'),np.load('refined_train_fc6_new.npy')],axis=1)
general_train = np.concatenate([np.load('general_train_fc10.npy'),np.load('general_train_fc6.npy')],axis=1)
general_val = np.concatenate([np.load('general_val_fc10.npy'),np.load('general_val_fc6.npy')],axis=1)
core_test = np.concatenate([np.load('core_test_fc10.npy'),np.load('core_test_fc6.npy')],axis=1)

In [55]:
# train_datasets = np.concatenate([general_train,general_val,refined_train,refined_val],axis=0)
train_datasets = np.concatenate([refined_train,refined_val],axis=0)

In [56]:
general_train_pred = pd.read_csv('general_train_pred.csv')['label'].to_numpy().astype('float32')
general_val_pred = pd.read_csv('general_val_pred.csv')['label'].to_numpy().astype('float32')
refined_train_pred = pd.read_csv('refined_train_pred.csv')['label'].to_numpy().astype('float32')
refined_val_pred = pd.read_csv('refined_val_pred.csv')['label'].to_numpy().astype('float32')
core_test_pred = pd.read_csv('core_test_pred.csv')['label'].to_numpy().astype('float32')

In [57]:
# train_pred = np.concatenate([general_train_pred,general_val_pred,refined_train_pred,refined_val_pred])
train_pred = np.concatenate([refined_train_pred,refined_val_pred])
train_pred_normed = train_pred/np.max(abs(train_pred))
core_test_pred_normed = core_test_pred/np.max(abs(core_test_pred))

In [58]:
train_dset = FusionDataset(numpy_file = train_datasets,
                    true_file = train_pred)
test_dset = FusionDataset(numpy_file = core_test,
                    true_file = core_test_pred)
train_loader = torch.utils.data.DataLoader(dataset=train_dset, 
                                           batch_size=batch_size, 
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dset, 
                                           batch_size=len(test_dset), 
                                           shuffle=False)

In [59]:
iter = 0
num_epochs=100
loss_min=100

for epoch in range(num_epochs):
    for i, (feat,labels) in enumerate(train_loader):
        
        # Load images with gradient accumulation capabilities

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        outputs = model(feat).reshape(-1)

        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        
        #scheduler.step()
        optimizer.step()
        
        iter += 1

        if iter % 1000 == 0:
            print('Iteration: {}. Loss: {}'.format(iter, loss.item()))
            if loss.item()<loss_min:
                loss_min = loss.item()
                save_model(epoch, model, optimizer, criterion)
            loss_ar.append(loss.item())
                
# Load from Checkpoint
checkpoint = torch.load('models/fusion.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# Test the Model

for i, (feat,labels) in enumerate(test_loader):
    outputs = model(feat).detach().numpy()
ytrue = labels
ypred = outputs.reshape(-1)
r2_score(ytrue,ypred),mean_absolute_error(ypred,ytrue),pearsonr(ypred,ytrue),spearmanr(ypred,ytrue),np.sqrt(mean_squared_error(ypred,ytrue))


Iteration: 1000. Loss: 5.804025650024414
Saving final model...
Iteration: 2000. Loss: 2.2687880992889404
Saving final model...
Iteration: 3000. Loss: 1.17572820186615
Saving final model...
Iteration: 4000. Loss: 1.5951635837554932
Iteration: 5000. Loss: 2.316443681716919
Iteration: 6000. Loss: 0.6069284677505493
Saving final model...
Iteration: 7000. Loss: 1.1931092739105225
Iteration: 8000. Loss: 1.3417868614196777
Iteration: 9000. Loss: 1.0254104137420654
Iteration: 10000. Loss: 1.6149123907089233


(0.525395990721731,
 1.1795211,
 PearsonRResult(statistic=0.7443099824474084, pvalue=1.5273503934223836e-51),
 SpearmanrResult(correlation=0.7401033390114357, pvalue=1.0984716465843504e-50),
 1.4953036)